In [ ]:
%matplotlib inline
%timeit
import sys
sys.path.append('pymodules')
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer

# this class read the raw input and tokenizes comprehensively for use with modeling
import pymodules.read_and_tokenize as contacts_utils

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import accuracy_score, mean_squared_error

from timeit import default_timer as timer

In [ ]:
filename = "data/Master-data_Q42021.xlsx"
prep_comments, df = contacts_utils.read_file(filename)

### Do we need bigrams ? Trigrams ?

In [ ]:
require_bigrams = True
if require_bigrams:
    for i in range(len(prep_comments.tokens)):
        prep_comments.tokens[i] = prep_comments.tokens[i] + prep_comments.bigrams[i]

test_index = 0
print(f"Comments at index[{test_index}] after addition of bigrams:\n {prep_comments.tokens[test_index]}")
print(f"Comments at index[{-1}] after addition of bigrams:\n {prep_comments.tokens[-1]}")

In [ ]:
len(prep_comments.tokens)

### Vectorization
* TF-IDF weights more words that occur frequently but in lesser number of documents. This seems to skew ranking towards advertisement like reviews. In any case for SVM, we use the count of tokens to establish a baseline accuracy of model, prior to using RNN model.

In [ ]:
# simple auxiliary function to override the preprocessing done by sklearn
def do_nothing(doc):
    return doc

# create a CountVectorizer object using our preprocessed text
# uni gram
count_vectorizer = CountVectorizer(encoding='utf-8',
                                   preprocessor=do_nothing,  # apply no additional preprocessing
                                   tokenizer=do_nothing,     # apply no additional tokenization
                                   lowercase=False,
                                   strip_accents=None,
                                   stop_words=None,
                                   ngram_range=(1, 1),       # generate only unigrams
                                   analyzer='word',          # analysis at the word-level
                                   #max_df=0.5,              # ignore tokens that have a higher document frequency (can be int or percent)
                                   #min_df=500,                # ignore tokens that have a lowe document frequency (can be int or percent)
                                   min_df=10,
                                   max_features=None,        # we could impose a maximum number of vocabulary terms
                                   )


In [ ]:
# transform our preprocessed tokens into a document-term matrix
dt_matrix = count_vectorizer.fit_transform(prep_comments.tokens)
print(f"Document-term matrix created with shape: {dt_matrix.shape}")

In [ ]:
# we can access a dictionary that maps between words and positions of the document-term matrix. We need this for SVM in order to make the word itself as column of dataframe (see output)
id_word_indexer = pd.DataFrame(count_vectorizer.vocabulary_.items())
id_word_indexer

In [ ]:
## FOR SVM, we need to make a matrix with proper column names
# Also, we need another column that denotes the review
# We also need to normalize the data
df_svm = pd.DataFrame(dt_matrix.toarray())
df_svm.rename(columns=id_word_indexer.to_dict()[0], inplace=True)
df_svm

In [ ]:
df

In [ ]:
# attach sentiment, seems
def find_sentiment(rating):
    choices = [0, 1, 2]
    conditions = [rating < 3, rating == 3, rating > 3]
    senti = np.select(conditions, choices)
    return senti

SENTIMENT_SERIES = df['RATING'].apply(find_sentiment).astype('category')
df_svm['_SENTIMENT_'] = SENTIMENT_SERIES.values
df_svm['_SENTIMENT_'] = df_svm['_SENTIMENT_'].astype('category')
df_svm._SENTIMENT_.value_counts()

In [ ]:
X = df_svm.drop(['_SENTIMENT_'], axis=1)
y = df_svm['_SENTIMENT_']
validation_reqd = True
df_trainX, df_trainy, df_testX, df_testy, df_validX, df_validy = contacts_utils.split_data(X, y, validation=True)

### Null accuracy
This is the accuracy based on the proportion of input distribution of sentiment values

In [ ]:
rating_counts = y.value_counts()
dn = np.sum(rating_counts)

null_rating_lt_3 = round((rating_counts[0]/dn)* 100, 3)
null_rating_eq_3 = round((rating_counts[1]/dn)* 100, 3)
null_rating_gt_3 = round((rating_counts[2]/dn)* 100, 3)
print(f"Null accuracy of sentiment rating < 3, corresponding to code 0 is: {null_rating_lt_3}%")
print(f"Null accuracy of sentiment rating = 3, corresponding to code 0 is: {null_rating_eq_3}%")
print(f"Null accuracy of sentiment rating > 3, corresponding to code 0 is: {null_rating_gt_3}%")


### Normalize column data please ...

In [ ]:
ss = StandardScaler()
df_trainX = ss.fit_transform(df_trainX)
df_testX = ss.fit_transform(df_testX)
df_validX = ss.fit_transform(df_validX)

In [ ]:
# https://towardsdatascience.com/hyperparameter-tuning-for-support-vector-machines-c-and-gamma-parameters-6a5097416167
# How to choose C and gamma values ...
# 0.0001 < gamma < 10
# 0.1 < C < 100
# When gamma is large, C does not matter ...

def create_svm_model(trainX, trainy, validX, validy):
    def _create_svm_model(param_dict):
        C = param_dict['C']
        kernel = param_dict['kernel']
        degree = param_dict['degree']

        model = SVC(C=C,
                        kernel=kernel,
                        degree=degree,
                        gamma='auto',
                        class_weight='balanced',
                        random_state=42)


        # fit model ...
        history = model.fit(X=trainX, y=trainy)
            #, validation_data = (validX,validy),
            #                verbose=1,
            #                workers=4,
            #                use_multiprocessing=True)

        y_pred_train = model.predict(trainX)#, workers=4, use_multiprocessing=True)
        y_pred_validation = model.predict(validX)#, workers=4, use_multiprocessing=True)

        #y_train_pred_labels = convert_prob_to_labels(y_pred_train)
        #y_validation_pred_labels = convert_prob_to_labels(y_pred_validation)

        # accuracy score used ...
        train_score = accuracy_score(trainy, y_pred_train)
        val_score = accuracy_score(validy, y_pred_validation)

        return train_score, val_score, param_dict, history, y_pred_validation

    return _create_svm_model

In [ ]:
# parameters to be varied
C = [1.0] #np.logspace(0, 1, 21)
kernel_functions = ['linear'] #('linear', 'poly', 'rbf', 'sigmoid')
degree_vals = [1]

parameters = [{'C': C,
              'kernel': kernel_functions,
              'degree': degree_vals
              }
              ]

# make a grid out of parameter choices ...
grid_params = ParameterGrid(parameters)
[print(x) for x in grid_params]

In [ ]:
# func that sets up the context .. i.e. what the pipeline does, what the data input is
svm_model_func = create_svm_model(df_trainX, df_trainy, df_validX, df_validy)

st_ = timer()
# run NN model in parallel and extract results (train_score, valid_score,parameter, history of fit) as a list
results = contacts_utils.run_parallel(svm_model_func, num_cpus=4)(grid_params)

end_ = timer()

print(f"Time taken to finish best parameter search with SVM model: {(end_-st_)/60.0} mins.")

### Why SVM ? Why not LR, Naive Bayes as NULL model ?
* LR ans SVM are closesly related. If the idea is to know the probabilities, then LR is a good option. But if the idea is to make the *right decision* (i.e. can be expressed as ratio of likelihoods), we end up with SVM method!
* Naive Bayes assumes *class conditional independence*. In this context, this means, given the sentiments, the features found are independent of each other. This i not true for this problem.
* LDA is very closely related to Naive Bayes with additional assumption of Gaussian distribution of features. So we can ignore this as well.
* What about decision trees and its friends ?
    * No proabilistic distribution assumption on response or features
    * Can be used in offline mode, but with addition of new data, one needs to retrain from scratch

In [ ]:
results

In [ ]:
# find the best parameters based on validation score
best_validation_score = results[0][1]
best_params = results[0][2]
plot_data = results[0][3]
best_validation_data = results[0][4]
for i in range(1, len(results)):
    tscore = results[i][0]
    vscore = results[i][1]
    param = results[i][2]
    if vscore > best_validation_score:
        best_validation_score = vscore
        best_params = param
        plot_data = results[i][3]
        best_validation_data = results[i][4]

# output result
print(f"Best validation score:{best_validation_score}")
print(f"Best params based on validation score:{best_params}")

In [ ]:
from sklearn import metrics
targetnames=['negative', 'neutral', 'positive']
#y_pred =   np.argmax(y_pred_raw, axis = 1)
#y_true = np.argmax(y_test, axis = 1)
print(metrics.classification_report(df_validy, best_validation_data, target_names=targetnames))

In [ ]:
# multi-label plot ...
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(df_validy, best_validation_data)
print(cm)
cm_display = ConfusionMatrixDisplay(cm).plot()
plt.show()

### Multi label stats
This function extracts Accuracy, Sensitivity, Specificity and Likelihood ratio for each of the 3 sentiment labels

In [ ]:
accuracy, sensitivity, specificity, likelihood = contacts_utils.model_stats_all_labels(best_validation_data, df_validy)
print(f'Accuracy of model(<3, = 3, >3) is:   {np.round(accuracy*100, 2)}%')
print(f'Sensitivity of the test(<3, = 3, >3):{np.round(sensitivity*100,2)}%')
print(f'Specificity of the test(<3, = 3, >3):{np.round(specificity*100,2)}%')
print(f'Likelihood ratio(<3, = 3, >3):       {np.round(likelihood, 2)}')

In [ ]:
# We extract the best params from above cells
best_model_params = best_params

# Instead of joining validation data and train data, we simply reload the original data and split then into
# train and test. This is suboptimal, but being relatively fast, this is ok for now
validation_reqd = False
# we want the output to be onehot encoded and this makes it easier to use other activation functions
# other than softmax for the last layer, (as this is a multi-class classification problem)
oneh = True
trainX, trainy, testX, testy, validX, validy = contacts_utils.split_data(X, y, validation_reqd)
print('Train: X=%s, y=%s' % (trainX.shape, trainy.shape))
print('Test: X=%s, y=%s' % (testX.shape, testy.shape))

# Now run the model with the best params
best_model_nn = create_svm_model(trainX, trainy, testX, testy)
st_ = timer()
# run NN model in parallel and extract results (train_score, valid_score,parameter, history of fit) as a list
b_train_score, b_val_score, b_param_dict, b_history, y_labels_predicted = best_model_nn(best_model_params)
end_ = timer()

accuracy, sensitivity, specificity, likelihood = contacts_utils.model_stats_all_labels(y_labels_predicted, testy)
print(f'Accuracy of model(<3, = 3, >3) is:   {np.round(accuracy*100, 2)}%')
print(f'Sensitivity of the test(<3, = 3, >3):{np.round(sensitivity*100,2)}%')
print(f'Specificity of the test(<3, = 3, >3):{np.round(specificity*100,2)}%')
print(f'Likelihood ratio(<3, = 3, >3):       {np.round(likelihood, 2)}')